In [ ]:
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [ ]:
df = pd.read_excel("Data Engineer Task.xlsx", engine='openpyxl', usecols='A', header=None)
# df

In [ ]:
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract


def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file,poppler_path=r"ADD POPPLER PATH HERE")


def ocr_core(file):
    pytesseract.pytesseract.tesseract_cmd = r'ADD TESSERACT PATH HERE'
    text = pytesseract.image_to_string(file,lang='mar')
    return text


def print_pages(pdf_file):
    images = pdf_to_img(pdf_file)
    text=[]
    for pg, img in enumerate(images):
        # print(ocr_core(img))
        text.append(ocr_core(img))
    
    return text

def download_file(url):
    local_filename = url.split('/')[-1]

    with requests.get(url) as r:
        assert r.status_code == 200, f'error, status code is {r.status_code}'
        with open(local_filename,'wb') as f:
            f.write(r.content)

    return local_filename

In [ ]:
def extract_text(url):
    page=url
    source = urlopen(page).read()
    soup = BeautifulSoup(source,'html.parser')
    link=soup.findAll('div', {'class': 'format-group'})[1].a['href']

    src = urlopen("https://archive.org"+link).read()
    text = BeautifulSoup(src,'html.parser')
    return text.pre

In [ ]:
file=[]

In [ ]:
for url in df[0]:
    dict={}
    dict['pdf_url'] = url
    if(url[-3:]!='pdf'):
        try:
            text=extract_text(url)
            dict['pdf_content']=text
        except:
            dict['pdf_content']=""
    else:
        text=print_pages(download_file(url))
        dict['pdf_content']=text
    
    file.append(dict)

In [ ]:
import json

In [ ]:
with open("pdf_extract.json", "w", encoding="utf-8") as outfile:
    json.dump(file, outfile, ensure_ascii=False)